In [1]:
#####################
# Train & Eval
#####################

# coding=utf-8
# Copyright 2021 The Google Research Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Lint as: python3
"""Trains TFT based on a defined set of parameters.

Uses default parameters supplied from the configs file to train a TFT model from
scratch.

Usage:
python3 script_train_fixed_params {expt_name} {output_folder}

Command line args:
  expt_name: Name of dataset/experiment to train.
  output_folder: Root folder in which experiment is saved


"""

import argparse
import datetime as dte
import os

import data_formatters.base
import expt_settings.configs
import libs.hyperparam_opt
import libs.tft_model
import libs.utils as utils
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf

import tensorflow
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))

ExperimentConfig = expt_settings.configs.ExperimentConfig
HyperparamOptManager = libs.hyperparam_opt.HyperparamOptManager
ModelClass = libs.tft_model.TemporalFusionTransformer
tf.experimental.output_all_intermediates(True)

2023-10-15 04:36:15.570709: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-15 04:36:15.595832: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available:  1


2023-10-15 04:36:16.691814: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-15 04:36:16.695581: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-15 04:36:16.695707: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
def main_train(expt_name,
         use_gpu,
         model_folder,
         data_csv_path,
         data_formatter,
         use_testing_mode=False):
    """Trains tft based on defined model params.

    Args:
      expt_name: Name of experiment
      use_gpu: Whether to run tensorflow with GPU operations
      model_folder: Folder path where models are serialized
      data_csv_path: Path to csv file containing data
      data_formatter: Dataset-specific data fromatter (see
        expt_settings.dataformatter.GenericDataFormatter)
      use_testing_mode: Uses a smaller models and data sizes for testing purposes
        only -- switch to False to use original default settings
    """

    num_repeats = 1

    if not isinstance(data_formatter, data_formatters.base.GenericDataFormatter):
        raise ValueError(
            "Data formatters should inherit from" +
            "AbstractDataFormatter! Type={}".format(type(data_formatter)))

    # Tensorflow setup
    default_keras_session = tf.keras.backend.get_session()

    if use_gpu:
        tf_config = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

    else:
        tf_config = utils.get_default_tensorflow_config(tf_device="cpu")

    print("*** Training from defined parameters for {} ***".format(expt_name))

    print("Loading & splitting data...")
    raw_data = pd.read_csv(data_csv_path, index_col=0)
    train, valid, test = data_formatter.split_data(raw_data)
    train_samples, valid_samples = data_formatter.get_num_samples_for_calibration()

    # Sets up default params
    fixed_params = data_formatter.get_experiment_params()
    params = data_formatter.get_default_model_params()
    params["model_folder"] = model_folder

    # Parameter overrides for testing only! Small sizes used to speed up script.
    if use_testing_mode:
        fixed_params["num_epochs"] = 1
        params["hidden_layer_size"] = 5
        train_samples, valid_samples = 100, 10

    # Sets up hyperparam manager
    print("*** Loading hyperparm manager ***")
    opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                       fixed_params, model_folder)


    # Training -- one iteration only
    print("*** Running calibration ***")
    print("Params Selected:")
    for k in params:
        print("{}: {}".format(k, params[k]))

    best_loss = np.Inf
    for _ in range(num_repeats):

        tf.reset_default_graph()
        with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

            tf.keras.backend.set_session(sess)

            params = opt_manager.get_next_parameters()
            model = ModelClass(params, use_cudnn=use_gpu)

            if not model.training_data_cached():
                model.cache_batched_data(train, "train", num_samples=train_samples)
                model.cache_batched_data(valid, "valid", num_samples=valid_samples)

            sess.run(tf.global_variables_initializer())
            model.fit()

            val_loss = model.evaluate()

            if val_loss < best_loss:
                opt_manager.update_score(params, val_loss, model)
                best_loss = val_loss

            tf.keras.backend.set_session(default_keras_session)

    print("*** Running tests ***")
    tf.reset_default_graph()
    with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:
        tf.keras.backend.set_session(sess)
        best_params = opt_manager.get_best_params()
        model = ModelClass(best_params, use_cudnn=use_gpu)

        model.load(opt_manager.hyperparam_folder)

        print("Computing best validation loss")
        val_loss = model.evaluate(valid)

        print("Computing test loss")
        output_map = model.predict(test, return_targets=True)
        targets = data_formatter.format_predictions(output_map["targets"])
        p50_forecast = data_formatter.format_predictions(output_map["p50"])
        p90_forecast = data_formatter.format_predictions(output_map["p90"])

        def extract_numerical_data(data):
            """Strips out forecast time and identifier columns."""
            return data[[
                col for col in data.columns
                if col not in {"forecast_time", "identifier"}
            ]]

        p50_loss = utils.numpy_normalised_quantile_loss(
            extract_numerical_data(targets), extract_numerical_data(p50_forecast),
            0.5)
        p90_loss = utils.numpy_normalised_quantile_loss(
            extract_numerical_data(targets), extract_numerical_data(p90_forecast),
            0.9)

        tf.keras.backend.set_session(default_keras_session)

    print("Training completed @ {}".format(dte.datetime.now()))
    print("Best validation loss = {}".format(val_loss))
    print("Params:")

    for k in best_params:
        print(k, " = ", best_params[k])
    print()
    print("Normalised Quantile Loss for Test Data: P50={}, P90={}".format(
        p50_loss.mean(), p90_loss.mean()))
    
    return output_map



In [3]:
dataset_name = 'crypto_day'
output_folder = '.'
use_tensorflow_with_gpu = 'yes'

print("Using output folder {}".format(output_folder))

config = ExperimentConfig(dataset_name, output_folder)
formatter = config.make_data_formatter()

# Customise inputs to main() for new datasets.
output_map = main_train(
    expt_name=dataset_name,
    use_gpu=use_tensorflow_with_gpu,
    model_folder=os.path.join(config.model_folder, "fixed"),
    data_csv_path=config.data_csv_path,
    data_formatter=formatter,
    use_testing_mode=False)  # Change to false to use original default params

Using output folder .


2023-10-15 04:36:16.729296: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-15 04:36:16.729433: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-15 04:36:16.729529: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Selecting GPU ID=0
*** Training from defined parameters for crypto_day ***
Loading & splitting data...
Formatting train-valid-test splits.
Setting scalers with training data...
*** Loading hyperparm manager ***
*** Running calibration ***
Params Selected:
dropout_rate: 0.3
hidden_layer_size: 160
learning_rate: 0.01
minibatch_size: 32
max_gradient_norm: 0.01
num_heads: 1
stack_size: 1
model_folder: ./saved_models/crypto_day/fixed
+++++++++++++++++++++++++++++++
{'dropout_rate': 0.3, 'hidden_layer_size': 160, 'learning_rate': 0.01, 'max_gradient_norm': 0.01, 'minibatch_size': 32, 'model_folder': './saved_models/crypto_day/fixed', 'num_heads': 1, 'stack_size': 1, 'total_time_steps': 37, 'num_encoder_steps': 30, 'num_epochs': 100, 'early_stopping_patience': 5, 'multiprocessing_workers': 5, 'column_definition': [('btc_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('btc_dt', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('btc_row_id', <DataTypes.REAL_VALUED: 0>, <InputTypes.STATIC_INPU

2023-10-15 04:36:17.306077: W tensorflow/c/c_api.cc:304] Operation '{name:'TemporalFusionTransformer/strided_slice_2/stack_1' id:12 op device:{requested: '', assigned: ''} def:{{{node TemporalFusionTransformer/strided_slice_2/stack_1}} = Const[_has_manual_control_dependencies=true, dtype=DT_INT32, value=Tensor<type: int32 shape: [3] values: 0 1 1>]()}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-10-15 04:36:17.309060: W tensorflow/c/c_api.cc:304] Operation '{name:'TemporalFusionTransformer/strided_slice_2/stack_2' id:13 op device:{requested: '', assigned: ''} def:{{{node TemporalFusionTransformer/strided_slice_2/stack_2}} = Const[_has_manual_control_dependencies=true, dtype=DT_INT32, value=Tensor<type: int32 shape: [3] values: 1 1 1>]()}}' was changed by setting attribute after it was run by a session. This mutation 

2023-10-15 04:36:17.508387: W tensorflow/c/c_api.cc:304] Operation '{name:'TemporalFusionTransformer/time_distributed_7/kernel/Assign' id:341 op device:{requested: '', assigned: ''} def:{{{node TemporalFusionTransformer/time_distributed_7/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](TemporalFusionTransformer/time_distributed_7/kernel, TemporalFusionTransformer/time_distributed_7/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-10-15 04:36:17.514688: W tensorflow/c/c_api.cc:304] Operation '{name:'TemporalFusionTransformer/strided_slice_11/stack_1' id:359 op device:{requested: '', assigned: ''} def:{{{node TemporalFusionTransformer/strided_slice_11/stack_1}} = Const[_has_manual_control_dependencies=true, dtype=D

...

INFO:tensorflow:Restoring parameters from ./saved_models/crypto_day/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss


/usr/local/lib/python3.8/dist-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2023-10-15 04:37:31.487771: W tensorflow/c/c_api.cc:304] Operation '{name:'TemporalFusionTransformer/loss/mul' id:10260 op device:{requested: '', assigned: ''} def:{{{node TemporalFusionTransformer/loss/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](TemporalFusionTransformer/loss/mul/x, TemporalFusionTransformer/loss/time_distributed_181_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Computing test loss


/usr/local/lib/python3.8/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-10-15 04:37:32.308397: W tensorflow/c/c_api.cc:304] Operation '{name:'TemporalFusionTransformer/time_distributed_181/Reshape_1' id:10175 op device:{requested: '', assigned: ''} def:{{{node TemporalFusionTransformer/time_distributed_181/Reshape_1}} = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _has_manual_control_dependencies=true](TemporalFusionTransformer/time_distributed_181/dense_211/BiasAdd, TemporalFusionTransformer/time_distributed_181/Reshape_1/shape)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Training completed @ 2023-10-15 04:37:33.156864
Best validation loss = 0.10117687911975502
Params:
dropout_rate  =  0.3
hidden_layer_size  =  160
learning_rate  =  0.01
max_gradient_norm  =  0.01
minibatch_size  =  32
model_folder  =  ./saved_models/crypto_day/fixed
num_heads  =  1
stack_size  =  1
total_time_steps  =  37
num_encoder_steps  =  30
num_epochs  =  100
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('btc_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('btc_dt', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('btc_row_id', <DataTypes.REAL_VALUED: 0>, <InputTypes.STATIC_INPUT: 3>), ('btc_low', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('btc_high', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('btc_open', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('btc_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('btc_volume', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INP

In [4]:
output_map

{'p10':    forecast_time identifier       t+0       t+1       t+2       t+3       t+4  \
 0     2023-09-19   BTC_USDT -0.731331 -0.734420 -0.770922 -0.770627 -0.771191   
 1     2023-09-20   BTC_USDT -0.736974 -0.770918 -0.770803 -0.771338 -0.792608   
 2     2023-09-21   BTC_USDT -0.771151 -0.770346 -0.771412 -0.792760 -0.789263   
 3     2023-09-22   BTC_USDT -0.770461 -0.770923 -0.792943 -0.789495 -0.795084   
 4     2023-09-23   BTC_USDT -0.770996 -0.792832 -0.789897 -0.795508 -0.785051   
 5     2023-09-24   BTC_USDT -0.791457 -0.789292 -0.795655 -0.785275 -0.741974   
 6     2023-09-25   BTC_USDT -0.788032 -0.795167 -0.785428 -0.742104 -0.748654   
 7     2023-09-26   BTC_USDT -0.793490 -0.784651 -0.741965 -0.748666 -0.744909   
 8     2023-09-27   BTC_USDT -0.783666 -0.740671 -0.748412 -0.744884 -0.678966   
 9     2023-09-28   BTC_USDT -0.742677 -0.747872 -0.744766 -0.678922 -0.710268   
 10    2023-09-29   BTC_USDT -0.749557 -0.744106 -0.678492 -0.710200 -0.715637   
 11    20

In [5]:
targets = formatter.format_predictions(output_map["targets"])
targets

,forecast_time,identifier,t+0,t+1,t+2,t+3,t+4,t+5,t+6
0,2023-09-19,BTC_USDT,26573.81,26578.04,26567.19,26246.48,26303.68,26218.78,26367.41
1,2023-09-20,BTC_USDT,26578.04,26567.19,26246.48,26303.68,26218.78,26367.41,27017.99
2,2023-09-21,BTC_USDT,26567.19,26246.48,26303.68,26218.78,26367.41,27017.99,26906.87
3,2023-09-22,BTC_USDT,26246.48,26303.68,26218.78,26367.41,27017.99,26906.87,26963.34
4,2023-09-23,BTC_USDT,26303.68,26218.78,26367.41,27017.99,26906.87,26963.34,27963.19
5,2023-09-24,BTC_USDT,26218.78,26367.41,27017.99,26906.87,26963.34,27963.19,27498.70
6,2023-09-25,BTC_USDT,26367.41,27017.99,26906.87,26963.34,27963.19,27498.70,27426.56
7,2023-09-26,BTC_USDT,27017.99,26906.87,26963.34,27963.19,27498.70,27426.56,27782.28
8,2023-09-27,BTC_USDT,26906.87,26963.34,27963.19,27498.70,27426.56,27782.28,27406.73
9,2023-09-28,BTC_USDT,26963.34,27963.19,27498.70,27426.56,27782.28,27406.73,27930.84


In [6]:
p10_forecast = formatter.format_predictions(output_map["p10"])
p10_forecast

,forecast_time,identifier,t+0,t+1,t+2,t+3,t+4,t+5,t+6
0,2023-09-19,BTC_USDT,24660.441406,24617.271484,24107.068359,24111.191406,24103.318359,23805.392578,23853.015625
1,2023-09-20,BTC_USDT,24581.574219,24107.126953,24108.740234,24101.261719,23803.968750,23852.287109,23772.771484
2,2023-09-21,BTC_USDT,24103.875000,24115.121094,24100.230469,23801.832031,23850.718750,23771.808594,23914.919922
3,2023-09-22,BTC_USDT,24113.515625,24107.056641,23799.273438,23847.472656,23769.359375,23913.531250,24514.583984
4,2023-09-23,BTC_USDT,24106.037109,23800.830078,23841.859375,23763.429688,23909.583984,24513.048828,24419.509766
5,2023-09-24,BTC_USDT,23820.046875,23850.314453,23761.369141,23906.464844,24511.683594,24419.035156,24470.992188
6,2023-09-25,BTC_USDT,23867.921875,23768.197266,23904.316406,24509.867188,24418.322266,24470.759766,25392.298828
7,2023-09-26,BTC_USDT,23791.628906,23915.183594,24511.816406,24418.148438,24470.664062,25392.218750,24954.832031
8,2023-09-27,BTC_USDT,23928.945312,24529.892578,24421.699219,24471.003906,25392.361328,24954.744141,24879.763672
9,2023-09-28,BTC_USDT,24501.857422,24429.244141,24472.658203,25392.976562,24954.837891,24879.687500,25210.779297


In [7]:
p50_forecast = formatter.format_predictions(output_map["p50"])
p50_forecast

,forecast_time,identifier,t+0,t+1,t+2,t+3,t+4,t+5,t+6
0,2023-09-19,BTC_USDT,25215.152344,25126.878906,24549.335938,24550.812500,24541.556641,24187.119141,24245.187500
1,2023-09-20,BTC_USDT,25125.509766,24559.949219,24548.031250,24538.410156,24184.798828,24243.898438,24145.630859
2,2023-09-21,BTC_USDT,24605.654297,24568.376953,24538.033203,24181.353516,24241.283203,24143.910156,24319.574219
3,2023-09-22,BTC_USDT,24616.498047,24559.148438,24178.367188,24235.906250,24139.671875,24317.142578,24993.019531
4,2023-09-23,BTC_USDT,24608.017578,24191.265625,24227.433594,24129.417969,24310.376953,24990.679688,24889.513672
5,2023-09-24,BTC_USDT,24272.912109,24252.458984,24127.240234,24305.271484,24988.636719,24888.816406,24946.216797
6,2023-09-25,BTC_USDT,24331.121094,24149.750000,24302.958984,24986.130859,24887.916016,24945.910156,26017.437500
7,2023-09-26,BTC_USDT,24237.724609,24331.753906,24990.447266,24888.017578,24945.914062,26017.416016,25506.197266
8,2023-09-27,BTC_USDT,24403.736328,25027.576172,24894.732422,24946.761719,26017.732422,25506.105469,25417.552734
9,2023-09-28,BTC_USDT,25036.664062,24916.455078,24950.406250,26018.759766,25506.279297,25417.478516,25806.757812


In [8]:
p90_forecast = formatter.format_predictions(output_map["p90"])
p90_forecast

,forecast_time,identifier,t+0,t+1,t+2,t+3,t+4,t+5,t+6
0,2023-09-19,BTC_USDT,29468.085938,29373.388672,28768.222656,28768.494141,28759.376953,28432.880859,28483.125000
1,2023-09-20,BTC_USDT,29381.708984,28781.533203,28765.632812,28755.876953,28430.298828,28481.707031,28396.400391
2,2023-09-21,BTC_USDT,28843.648438,28789.587891,28755.683594,28426.521484,28478.806641,28394.515625,28549.500000
3,2023-09-22,BTC_USDT,28854.119141,28780.449219,28423.437500,28472.873047,28389.833984,28546.806641,29232.140625
4,2023-09-23,BTC_USDT,28845.791016,28438.757812,28463.800781,28378.539062,28539.332031,29229.515625,29123.349609
5,2023-09-24,BTC_USDT,28542.033203,28492.335938,28376.396484,28533.740234,29227.234375,29122.568359,29183.716797
6,2023-09-25,BTC_USDT,28591.957031,28402.169922,28531.466797,29224.425781,29121.570312,29183.367188,30210.107422
7,2023-09-26,BTC_USDT,28512.460938,28564.115234,29229.546875,29121.685547,29183.400391,30210.113281,29727.664062
8,2023-09-27,BTC_USDT,28655.880859,29273.027344,29129.437500,29184.343750,30210.472656,29727.593750,29643.912109
9,2023-09-28,BTC_USDT,29292.933594,29155.669922,29188.720703,30211.660156,29727.800781,29643.857422,30007.449219


In [10]:
target_df = pd.read_csv("./data/crypto_day/crypto_day.csv")
target_df = target_df.loc[target_df['btc_dt']>'2023-09-19']
target_df[['btc_dt','target']]

,btc_dt,target
1053,2023-09-20,26573.81
1054,2023-09-21,26578.04
1055,2023-09-22,26567.19
1056,2023-09-23,26246.48
1057,2023-09-24,26303.68
1058,2023-09-25,26218.78
1059,2023-09-26,26367.41
1060,2023-09-27,27017.99
1061,2023-09-28,26906.87
1062,2023-09-29,26963.34
